In [1]:
# Uncomment this line if you need to install the dependencies
#!pip install requests pystac-client cogeo-mosaic==7.1.0 aiofiles httpx

In [1]:
from pystac_client import Client
import requests
import json
from cogeo_mosaic.mosaic import MosaicJSON

In [2]:
stac_api_url = "https://api.impactobservatory.com/stac-aws"
collection_name = "io-10m-annual-lulc"

titiler_url = "https://tiler.staging.modelmywatershed.org"

In [3]:
mosaics = {}

# For each year
# 1. search for the STAC Items for that year
# 2. get the S3 URI for the asset
# 3. compute a mosaicjson for it
# 4. create that mosaicjson in titiler-mosaicjson

for year in ["2017", "2018", "2019", "2020", "2021", "2022", "2023"]:
  search = Client.open(stac_api_url).search(
    limit=1000,
    collections=[collection_name],
    datetime=[f"{year}-06-01T00:00:00Z", f"{year}-06-02T00:00:00Z"]
  )
  
  asset_hrefs = list(item["assets"]["supercell"]["href"] for item in search.items_as_dicts())
  mosaic_definition = MosaicJSON.from_urls(asset_hrefs)
  mosaic_definition.name = f"{year} Impact Observatory 10m Annual Land Use Land Cover (9-class)"
  # print(json.dumps(mosaic_definition.model_dump(), indent=2))

  r = requests.post(
    url=f"{titiler_url}/mosaicjson/mosaics",
    headers={
      "Content-Type": "application/vnd.titiler.mosaicjson+json",
    },
    json=mosaic_definition.model_dump()
  )

  response_body = r.json()

  tiles_link_href = next((link["href"] for link in response_body["links"] if link["rel"] == "tiles"), None)

  mosaics[year] = tiles_link_href

  print(f"{year}: {tiles_link_href}")

print(json.dumps(mosaics, indent=2))


/Users/philvarner/code/mmw/mmw-tiler/.venv/lib/python3.11/site-packages/cogeo_mosaic/mosaic.py:283: UserWarning: Multiple MinZoom, Assets different minzoom values
  warnings.warn(
/Users/philvarner/code/mmw/mmw-tiler/.venv/lib/python3.11/site-packages/cogeo_mosaic/mosaic.py:292: UserWarning: Multiple MaxZoom, Assets have multiple resolution values
  warnings.warn(


2017: https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/648cfb08-d7a2-4351-80b3-5c6505fc1ce3/tiles/{z}/{x}/{y}
{
  "2017": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/648cfb08-d7a2-4351-80b3-5c6505fc1ce3/tiles/{z}/{x}/{y}"
}


In [ ]:
COLORMAP = "%7B%220%22%3A%20%22%23000000%22%2C%20%221%22%3A%20%22%23419bdf%22%2C%20%222%22%3A%20%22%23397d49%22%2C%20%223%22%3A%20%22%23000000%22%2C%20%224%22%3A%20%22%237a87c6%22%2C%20%225%22%3A%20%22%23e49635%22%2C%20%226%22%3A%20%22%23000000%22%2C%20%227%22%3A%20%22%23c4281b%22%2C%20%228%22%3A%20%22%23a59b8f%22%2C%20%229%22%3A%20%22%23a8ebff%22%2C%20%2210%22%3A%20%22%23616161%22%2C%20%2211%22%3A%20%22%23e3e2c3%22%7D"
pre_generated_mosaics = {
  "2017": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/6900be40-a0d8-407f-a5d5-cc431310686c/tiles/{z}/{x}/{y}",
  "2018": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/4a0dc777-6fcf-4a46-8e2e-fd0b97d712d8/tiles/{z}/{x}/{y}",
  "2019": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/570f1af6-049d-45bb-a31e-0b1a88524283/tiles/{z}/{x}/{y}",
  "2020": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/47e3dc4d-1849-4830-a1c2-3c9a3186644d/tiles/{z}/{x}/{y}",
  "2021": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/8f8316e7-6100-4c8b-bd4d-91dcd104ff09/tiles/{z}/{x}/{y}",
  "2022": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/d36543c2-6b4f-47a8-9b13-0341ce222a31/tiles/{z}/{x}/{y}",
  "2023": "https://tiler.staging.modelmywatershed.org/mosaicjson/mosaics/fec54b55-5e75-4c6d-ac47-41579d722dab/tiles/{z}/{x}/{y}"
}

tile_url_template = pre_generated_mosaics["2017"]

tile_layer_url = f"{tile_url_template}@2x.png?colormap={COLORMAP}"

print(f"Tile Layer URL for Leaflet: {tile_layer_url}")

For a given zoom level, download all of the tiles into a local directory
The local layout is like `/tiles/{mosaic_id}/tiles/{z}/{x}/{y}/{param_hash}`
e.g., `tiles/6900be40-a0d8-407f-a5d5-cc431310686c/tiles/4/0/0/f6914f4a30a75d202ad1ae24ea6c89961a995f0bfd65838dd5896e106fea1ec5`
the hash is used because there are two parameters (one path and one query param) that determine what the resulting tile
will will look like. The local path structure mimics the URL structure used for the tile of
`/mosaicjson/{mosaic_id}/tiles/{z}/{x}/{y}`
 added to these are parameters `{scale}{format}?colormap={colormap}`

- `scale` is one of empty string, `@1x` or `@2x`.
- `format` is always `.png` so no data will show as transparent
- `colormap` is a JSON object describing the data's digital value to color mapping

Before running this, the tiler Lambda needs to be manually edited to change two settings:

1. Enable the Lambda Function URL. The value for this should be populated into `tiler_lambda_function_url` below.
2. Edit the timeout setting to set it to 900 seconds (the max value).

In [5]:
import asyncio
import httpx
import aiofiles
import aiofiles.os
import hashlib
import os
from pathlib import Path



tiler_url = "https://tiler.staging.modelmywatershed.org"
tiler_lambda_function_url = "https://2empcsyg6p3r53jhbp6v2ovsia0awpdn.lambda-url.us-west-2.on.aws"

tile_url_template = tile_url_template.replace(tiler_url,tiler_lambda_function_url)
zoom_level = 4

async def download_tile(tile_url_template: str, z: int, x: int, y: int):
    tile_url = tile_url_template.replace("{z}", str(z)).replace("{x}", str(x)).replace("{y}", str(y))
    mosaic_path = tile_url.split("/mosaicjson/mosaics/", maxsplit=1)[1]
    for scale in ["", "@1x", "@2x"]:
        params = f"{scale}.png?colormap={COLORMAP}"
        tile_url = f"{tile_url}{params}"
        mosaic_hash = hashlib.sha256(params.encode()).hexdigest()
        local_path = f"tiles/{mosaic_path}/{mosaic_hash}"
        if not Path(local_path).exists():
            async with httpx.AsyncClient() as client:
                r = await client.get(tile_url, timeout=httpx.Timeout(5.0, read=900))
                match r.status_code:
                    case httpx.codes.OK:
                        await aiofiles.os.makedirs(f"tiles/{mosaic_path}", exist_ok=True)
                        async with aiofiles.open(local_path, "wb") as f:
                            await f.write(r.content)
                    case httpx.codes.NOT_FOUND:
                        await aiofiles.os.makedirs(f"tiles/{mosaic_path}", exist_ok=True)
                        await aiofiles.open(local_path, "w+")
                    case _:
                        return f"Error: {tile_url} => {r.status_code}: {r.text}"

async def download_tiles():
    zxy_tuples = [(zoom_level, x, y) for x in range(0, 2**zoom_level) for y in range(0, 2**zoom_level)]
    return await asyncio.gather(*(download_tile(tile_url_template, *args) for args in zxy_tuples))

os.makedirs("tiles", exist_ok=True)
result = await asyncio.get_event_loop().create_task(download_tiles())
print(f"Failures: {[r for r in result if r]}")

Failures: ['Error: https://2empcsyg6p3r53jhbp6v2ovsia0awpdn.lambda-url.us-west-2.on.aws/mosaicjson/mosaics/6900be40-a0d8-407f-a5d5-cc431310686c/tiles/4/0/0.png?colormap=%7B%220%22%3A%20%22%23000000%22%2C%20%221%22%3A%20%22%23419bdf%22%2C%20%222%22%3A%20%22%23397d49%22%2C%20%223%22%3A%20%22%23000000%22%2C%20%224%22%3A%20%22%237a87c6%22%2C%20%225%22%3A%20%22%23e49635%22%2C%20%226%22%3A%20%22%23000000%22%2C%20%227%22%3A%20%22%23c4281b%22%2C%20%228%22%3A%20%22%23a59b8f%22%2C%20%229%22%3A%20%22%23a8ebff%22%2C%20%2210%22%3A%20%22%23616161%22%2C%20%2211%22%3A%20%22%23e3e2c3%22%7D@1x.png?colormap=%7B%220%22%3A%20%22%23000000%22%2C%20%221%22%3A%20%22%23419bdf%22%2C%20%222%22%3A%20%22%23397d49%22%2C%20%223%22%3A%20%22%23000000%22%2C%20%224%22%3A%20%22%237a87c6%22%2C%20%225%22%3A%20%22%23e49635%22%2C%20%226%22%3A%20%22%23000000%22%2C%20%227%22%3A%20%22%23c4281b%22%2C%20%228%22%3A%20%22%23a59b8f%22%2C%20%229%22%3A%20%22%23a8ebff%22%2C%20%2210%22%3A%20%22%23616161%22%2C%20%2211%22%3A%20%22%23e3e2c3